In [1]:
import pandas as pd
import numpy as np
import statistics
import os

from tqdm import tqdm

In [2]:
fraction_list = ["p6", "p7", "p8", "p9"]
periodic_fracs = [f.replace('p', 'a') for f in fraction_list]

In [3]:
periodic_fracs

['a6', 'a7', 'a8', 'a9']

In [4]:
def extract_metrics(file_path, dataset, model):
    lines=None
    # print(f"HAHAH: {file_path}")
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # print(f"HAHAH 222: {file_path}")
    except:
        # print(e)
        # print(f"File: {file_path} does not exist")
        return None
        
    lines = [line.strip() for line in lines if line.strip()]
    # Only consider the last 8 lines (4 configurations + their metrics)
    lines = lines[-8:]
    # print(lines)
    metrics = []
    
    if model=='SimMTM' or model=='MissTSM':
        for i in range(0, len(lines), 1):
            metrics_line = lines[i].strip()
            mse_value = float(metrics_line.split(',')[1].strip())
            mae_value = float(metrics_line.split(',')[2].strip())
            
            config = metrics_line.split('->')[1].split(',')[0]
            metrics.append({
                'configuration' : config,
                'mse': mse_value,
                'mae': mae_value
            })
        return metrics
            
    for i in range(0, len(lines), 2):
        config_line = lines[i].strip().lower()
        metrics_line = lines[i + 1].strip()
        
        mse_value = float(metrics_line.split('mse:')[1].split(',')[0])
        mae_value = float(metrics_line.split('mae:')[1].split(',')[0])
        
        # print(config_line)
        if model=='PatchTST':
            config=config_line.split('336_')[1].split('_')[0]
            if config=='':#336
                config='336'
        else:
            config=config_line.split(dataset.lower()+'_336_')[1].split('_')[0]
        # print(config)
        metrics.append({
            'configuration': config,
            'mse': mse_value,
            'mae': mae_value
        })
    
    return metrics

In [7]:
import statistics

In [14]:
results = []
break_flag=0

imp_models_per_ts_model = {'DLinear':['SAITS', 'Spline'],
                     'Autoformer':['Spline'],
                     'iTransformer':['Spline'],
                     'PatchTST':['SAITS', 'Spline', 'kNN', 'BRITS'],
                     'SimMTM':['SAITS', 'Spline']}
dataset_per_ts_model = {'DLinear':['ETTh2', 'weather', 'traffic'],
                     'Autoformer':['ETTh1'],
                     'iTransformer':['ETTh1'],
                     'PatchTST':['weather'],
                     'SimMTM':['ETTh2', 'weather', 'traffic', 'ETTm2']}

dataset="ETTh1"
fraction_list=["p8", "p9"]

for ts_model in ['PatchTST']:
    print(f"\n\nTS MODEL = {ts_model}")

    sub_dir_path = '../iTransformer-main/outputs/Spline/mcar/'
    for fraction in tqdm(fraction_list):
        print(f"Fraction = {fraction}")
        row = [fraction]
        mse_trials = {'96':[], '192':[], '336':[], '720':[]}
        mae_trials = {'96':[], '192':[], '336':[], '720':[]}

        for trial in [0, 1, 2]:
            file_path = f"{sub_dir_path}/{dataset}_v{trial}/result_{fraction}.txt"
            metrics = extract_metrics(file_path, dataset, ts_model)

            for metric in metrics:
                mse_trials[metric['configuration']].append(metric['mse'])
                mae_trials[metric['configuration']].append(metric['mae'])
        
        for config in ['96','192','336','720']:
            mean=statistics.mean(mse_trials[config])
            std=statistics.stdev(mse_trials[config])
            print(f"For Fraction = {fraction}:: {config} = {mean}; +/- {std}")



TS MODEL = PatchTST


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 309.08it/s]

Fraction = p8
For Fraction = p8:: 96 = 0.7479339322624593; +/- 0.031997581219329024
For Fraction = p8:: 192 = 0.7646825641232198; +/- 0.030947260599318394
For Fraction = p8:: 336 = 0.7940592674403287; +/- 0.027498192581780777
For Fraction = p8:: 720 = 0.8354026883160949; +/- 0.0071529989424666774
Fraction = p9
For Fraction = p9:: 96 = 1.1217350815059244; +/- 0.20165638339017328
For Fraction = p9:: 192 = 1.1438893440844804; +/- 0.20683869286932557
For Fraction = p9:: 336 = 1.0906342467458794; +/- 0.19814368934564278
For Fraction = p9:: 720 = 0.9469103580002541; +/- 0.08956462736764444


In [27]:
cols = ['Fraction', 'Trial', 'Dataset', 'Masking', 'TS_Model', 'Prediction_Length', 'Imputation_Model', 'MSE', 'MAE']

In [28]:
final_df = pd.DataFrame(results, columns=cols)
final_df

,Fraction,Trial,Dataset,Masking,TS_Model,Prediction_Length,Imputation_Model,MSE,MAE
0,p6,0,weather,MCAR,PatchTST,96,SAITS,0.172300,0.226823
1,p6,0,weather,MCAR,PatchTST,192,SAITS,0.213874,0.267457
2,p6,0,weather,MCAR,PatchTST,336,SAITS,0.263275,0.306354
3,p6,0,weather,MCAR,PatchTST,720,SAITS,0.334317,0.357840
4,p6,1,weather,MCAR,PatchTST,96,SAITS,0.172742,0.227108
...,...,...,...,...,...,...,...,...,...
315,p9,3,weather,MCAR,PatchTST,720,BRITS,0.713374,0.647282
316,p9,4,weather,MCAR,PatchTST,96,BRITS,0.848817,0.681006
317,p9,4,weather,MCAR,PatchTST,192,BRITS,0.789759,0.665266
318,p9,4,weather,MCAR,PatchTST,336,BRITS,0.779503,0.665870


In [29]:
final_df.to_csv('patchtst_fe_trial.csv', index=0)

In [74]:
final_df.TS_Model.unique()

array(['DLinear', 'Autoformer', 'iTransformer', 'PatchTST', 'SimMTM'],
      dtype=object)

In [75]:
final_df.to_csv('baselines_updated.csv', index=0)

In [76]:
'''
MissTSM
'''

'\nMissTSM\n'

In [22]:
results_misstsm = []
break_flag=0
imputation_model = 'None'
imp_models_per_ts_model = {'DLinear':['SAITS', 'Spline'],
                     'Autoformer':['SAITS', 'Spline'],
                     'iTransformer':['SAITS', 'Spline', 'kNN', 'BRITS'],
                     'PatchTST':['SAITS', 'Spline', 'kNN', 'BRITS'],
                     'SimMTM':['SAITS', 'Spline']}
dataset_per_ts_model = {'MissTSM':['weather']}

for ts_model in ['MissTSM']:
    print(f"\n\nTS MODEL = {ts_model}")
    for dataset in dataset_per_ts_model[ts_model]:
        print(f"\nDataset = {dataset}")
        # if break_flag:
        #     break
        # if (ts_model != 'DLinear') and (dataset!="ETTh2"):
        #     dataset = dataset.lower()
        # if ts_model=='SimMTM' and dataset=='weather':
        #     continue
        
        # for imputation_model in imp_models_per_ts_model[ts_model]:
        #     print(f"Imputation model = {imputation_model}")
            # if break_flag:
            #     break
#             if (ts_model=='DLinear') and (dataset=='traffic') and (imputation_model=='SAITS'):
#                 break
            
        sub_dir_path = './' + ts_model
        # if not os.path.isdir(sub_dir_path):
        #     print(f"{imputation_model} does not exist for {ts_model}")
        #     continue
        # # if imputation_model=='BRITS' or imputation_model=='kNN':
        #     if dataset=='traffic' or dataset=='ETTm2':
        #         continue

        # if ts_model=='SimMTM' and imputation_model=='Spline' and dataset=='ETTm2':
        #     continue
        pth = f"./{ts_model}/mcar"
        if not os.path.exists(pth):
            print(f"{pth} does not exist")
            # break_flag=1
            continue

        for fraction in tqdm(fraction_list):

#                 if break_flag:
#                     break
            print(f"Fraction = {fraction}")
            row = [fraction]
            mse_trials = {'96':[], '192':[], '336':[], '720':[]}
            mae_trials = {'96':[], '192':[], '336':[], '720':[]}

            for trial in [0, 1, 2, 3, 4]:
                # file_path = f"./{ts_model}/{imputation_model}/mcar/"
                # if not os.path.exists(file_path):
                #     print(f"{file_path} does not exist")
                #     break_flag=1
                #     break
                # else:
                #     break_flag=0
                file_path = f"{pth}/{dataset}_v{trial}/result_{fraction}.txt"
                metrics = extract_metrics(file_path, dataset, ts_model)
                if metrics is None:
                    continue
                    # break_flag=1
                    # break
                # else:
                #     break_flag=0

                for metric in metrics:
                    # mse_trials[metric['configuration']].append(metric['mse'])
                    # mae_trials[metric['configuration']].append(metric['mae'])
                    
                    # if metric['configuration']=='720':
                    results_misstsm.append((fraction, trial, dataset, 'MCAR', ts_model, metric['configuration'], metric['mse'], metric['mae']))

#             if break_flag:
#                 # break_flag=1
#                 break
#             # get the average
#             # print(f"mamams:{mse_trials}")
#             for config in ['96', '192', '336', '720']:
#                 if len(mse_trials[config])<5:
#                     print(f"CONFIG={config}. Less number of trials mcar ({len(mse_trials[config])}) for {pth}")
#                     if len(mse_trials[config])<3:
#                         continue
#                 mse_mean = statistics.mean(mse_trials[config])
#                 mse_std = statistics.stdev(mse_trials[config])

#                 mae_mean = statistics.mean(mae_trials[config])
#                 mae_std = statistics.stdev(mae_trials[config])

#                 results_misstsm.append((fraction, dataset, 'MCAR', ts_model, config, imputation_model, mse_mean, mse_std, mae_mean, mae_std))

#         pth = f"./{ts_model}/periodic"
#         if not os.path.exists(pth):
#             print(f"{pth} does not exist")
#             # break_flag=1
#             continue

#         for fraction in tqdm(periodic_fracs):

# #                 if break_flag:
# #                     break
#             print(f"Fraction = {fraction}")
#             row = [fraction]
#             mse_trials = {'96':[], '192':[], '336':[], '720':[]}
#             mae_trials = {'96':[], '192':[], '336':[], '720':[]}

#             for trial in [0, 1, 2, 3, 4]:
#                 file_path = f"{pth}/{dataset}_v{trial}/result_{fraction}.txt"
#                 metrics = extract_metrics(file_path, dataset, ts_model)
#                 if metrics is None:
#                     # break_flag=1
#                     continue
#                 # else:
#                 #     break_flag=0

#                 for metric in metrics:
#                     mse_trials[metric['configuration']].append(metric['mse'])
#                     mae_trials[metric['configuration']].append(metric['mae'])

#             if break_flag:
#                 # break_flag=1
#                 break
#             # get the average
#             # print(f"periodic:{mse_trials}")
#             for config in ['96', '192', '336', '720']:
#                 # print()
#                 if config in mse_trials and len(mse_trials[config])<3:
#                     print(f"CONFIG={config}. Less number of trials periodic ({len(mse_trials[config])}) for {pth}")
#                     if len(mse_trials[config])<3:
#                         continue
#                 mse_mean = statistics.mean(mse_trials[config])
#                 mse_std = statistics.stdev(mse_trials[config])

#                 mae_mean = statistics.mean(mae_trials[config])
#                 mae_std = statistics.stdev(mae_trials[config])

#                 results_misstsm.append((fraction, dataset, 'Periodic', ts_model, config, imputation_model, mse_mean, mse_std, mae_mean, mae_std))



TS MODEL = MissTSM

Dataset = weather


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 2411.21it/s]

Fraction = p6
Fraction = p7
Fraction = p8
Fraction = p9


In [23]:
cols = ['Fraction', 'Trial', 'Dataset', 'Masking', 'TS_Model', 'Prediction_Length', 'MSE', 'MAE']

In [24]:
final_df_misstsm = pd.DataFrame(results_misstsm, columns=cols)
final_df_misstsm

,Fraction,Trial,Dataset,Masking,TS_Model,Prediction_Length,MSE,MAE
0,p6,0,weather,MCAR,MissTSM,96,0.191,0.258
1,p6,0,weather,MCAR,MissTSM,192,0.233,0.295
2,p6,0,weather,MCAR,MissTSM,336,0.284,0.327
3,p6,0,weather,MCAR,MissTSM,720,0.310,0.345
4,p6,1,weather,MCAR,MissTSM,96,0.191,0.263
5,p6,1,weather,MCAR,MissTSM,192,0.228,0.294
6,p6,1,weather,MCAR,MissTSM,336,0.281,0.320
7,p6,1,weather,MCAR,MissTSM,720,0.332,0.359
8,p6,2,weather,MCAR,MissTSM,96,0.193,0.261
9,p6,2,weather,MCAR,MissTSM,192,0.243,0.301


In [25]:
final_df_misstsm.to_csv('weather_misstsm.csv', index=0)

In [15]:
final_df_misstsm.to_csv('misstsm_weather_720_mcar.csv', index=False)

In [79]:
df1=pd.read_csv('baselines_updated.csv')
df1

,Fraction,Dataset,Masking,TS_Model,Prediction_Length,Imputation_Model,MSE_Mean,MSE_STD,MAE_Mean,MAE_STD
0,p6,ETTh2,MCAR,DLinear,96,SAITS,0.267415,0.015955,0.336383,0.008934
1,p6,ETTh2,MCAR,DLinear,192,SAITS,0.377993,0.031653,0.409279,0.015572
2,p6,ETTh2,MCAR,DLinear,336,SAITS,0.449674,0.036940,0.460009,0.018715
3,p6,ETTh2,MCAR,DLinear,720,SAITS,0.618890,0.046420,0.550535,0.020014
4,p7,ETTh2,MCAR,DLinear,96,SAITS,0.253664,0.028532,0.326745,0.018440
...,...,...,...,...,...,...,...,...,...,...
692,a8,ETTm2,Periodic,SimMTM,96,Spline,0.165800,0.004970,0.248200,0.004025
693,a8,ETTm2,Periodic,SimMTM,192,Spline,0.224200,0.005070,0.287200,0.003347
694,a8,ETTm2,Periodic,SimMTM,336,Spline,0.280800,0.005630,0.323200,0.004658
695,a8,ETTm2,Periodic,SimMTM,720,Spline,0.372200,0.006140,0.378400,0.004393


In [80]:
df1.append(final_df_misstsm, ignore_index=True).to_csv('all_models_v2.csv', index=0)

/tmp/ipykernel_67814/3688213192.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(final_df_misstsm, ignore_index=True).to_csv('all_models_v2.csv', index=0)


In [113]:
df1 = pd.read_csv('all_models_v2.csv')

In [99]:
df1.TS_Model.unique()

array(['DLinear', 'Autoformer', 'iTransformer', 'PatchTST', 'SimMTM',
       'MissTSM'], dtype=object)

In [108]:
'''
simmtm
'''

simmtm_etth2 = [('p0', 'ETTh2', 'None', 'SimMTM', '96', 'None', 0.295, -1.0, 0.350, -1.0),
               ('p0', 'ETTh2', 'None', 'SimMTM', '192', 'None', 0.356, -1.0, 0.389, -1.0),
               ('p0', 'ETTh2', 'None', 'SimMTM', '336', 'None', 0.375, -1.0, 0.407, -1.0),
               ('p0', 'ETTh2', 'None', 'SimMTM', '720', 'None', 0.404, -1.0, 0.437, -1.0)]

simmtm_ettm2 = [('p0', 'ETTm2', 'None', 'SimMTM', '96', 'None', 0.172, -1.0, 0.261, -1.0),
               ('p0', 'ETTm2', 'None', 'SimMTM', '192', 'None', 0.223, -1.0, 0.300, -1.0),
               ('p0', 'ETTm2', 'None', 'SimMTM', '336', 'None', 0.282, -1.0, 0.331, -1.0),
               ('p0', 'ETTm2', 'None', 'SimMTM', '720', 'None', 0.374, -1.0, 0.388, -1.0)]

simmtm_weather = [('p0', 'weather', 'None', 'SimMTM', '96', 'None', 0.158, -1.0, 0.211, -1.0),
               ('p0', 'weather', 'None', 'SimMTM', '192', 'None', 0.199, -1.0, 0.249, -1.0),
               ('p0', 'weather', 'None', 'SimMTM', '336', 'None', 0.246, -1.0, 0.286, -1.0),
               ('p0', 'weather', 'None', 'SimMTM', '720', 'None', 0.317, -1.0, 0.337, -1.0)]

In [109]:
'''
patchtst
'''

ptst_etth2 = [('p0', 'ETTh2', 'None', 'PatchTST', '96', 'None', 0.274, -1.0, 0.336, -1.0),
               ('p0', 'ETTh2', 'None', 'PatchTST', '192', 'None', 0.338, -1.0, 0.378, -1.0),
               ('p0', 'ETTh2', 'None', 'PatchTST', '336', 'None', 0.33, -1.0, 0.38, -1.0),
               ('p0', 'ETTh2', 'None', 'PatchTST', '720', 'None', 0.378, -1.0, 0.421, -1.0)]

ptst_ettm2 = [('p0', 'ETTm2', 'None', 'PatchTST', '96', 'None', 0.164, -1.0, 0.254, -1.0),
               ('p0', 'ETTm2', 'None', 'PatchTST', '192', 'None', 0.22, -1.0, 0.291, -1.0),
               ('p0', 'ETTm2', 'None', 'PatchTST', '336', 'None', 0.277, -1.0, 0.329, -1.0),
               ('p0', 'ETTm2', 'None', 'PatchTST', '720', 'None', 0.367, -1.0, 0.384, -1.0)]

ptst_weather = [('p0', 'weather', 'None', 'PatchTST', '96', 'None', 0.151, -1.0, 0.2, -1.0),
               ('p0', 'weather', 'None', 'PatchTST', '192', 'None', 0.196, -1.0, 0.242, -1.0),
               ('p0', 'weather', 'None', 'PatchTST', '336', 'None', 0.249, -1.0, 0.284, -1.0),
               ('p0', 'weather', 'None', 'PatchTST', '720', 'None', 0.319, -1.0, 0.334, -1.0)]

In [110]:
'''
itransformer
'''

itransformer_etth2 = [('p0', 'ETTh2', 'None', 'iTransformer', '96', 'None', 0.304, -1.0, 0.359, -1.0),
                      ('p0', 'ETTh2', 'None', 'iTransformer', '192', 'None', 0.392, -1.0, 0.412, -1.0),
                      ('p0', 'ETTh2', 'None', 'iTransformer', '336', 'None', 0.425, -1.0, 0.438, -1.0),
                      ('p0', 'ETTh2', 'None', 'iTransformer', '720', 'None', 0.415, -1.0, 0.442, -1.0)]

itransformer_weather = [('p0', 'weather', 'None', 'iTransformer', '96', 'None', 0.163, -1.0, 0.212, -1.0),
                        ('p0', 'weather', 'None', 'iTransformer', '192', 'None', 0.203, -1.0, 0.248, -1.0),
                        ('p0', 'weather', 'None', 'iTransformer', '336', 'None', 0.255, -1.0, 0.289, -1.0),
                        ('p0', 'weather', 'None', 'iTransformer', '720', 'None', 0.326, -1.0, 0.337, -1.0)]

In [111]:
'''
autoformer
'''

autoformer_etth2 = [('p0', 'ETTh2', 'None', 'Autoformer', '96', 'None', 0.501, -1.0, 0.522, -1.0),
                      ('p0', 'ETTh2', 'None', 'Autoformer', '192', 'None', 0.516, -1.0, 0.544, -1.0),
                      ('p0', 'ETTh2', 'None', 'Autoformer', '336', 'None', 0.565, -1.0, 0.573, -1.0),
                      ('p0', 'ETTh2', 'None', 'Autoformer', '720', 'None', 0.462, -1.0, 0.490, -1.0)]

autoformer_weather = [('p0', 'weather', 'None', 'Autoformer', '96', 'None', 0.306, -1.0, 0.380, -1.0),
                        ('p0', 'weather', 'None', 'Autoformer', '192', 'None', 0.434, -1.0, 0.474, -1.0),
                        ('p0', 'weather', 'None', 'Autoformer', '336', 'None', 0.437, -1.0, 0.453, -1.0),
                        ('p0', 'weather', 'None', 'Autoformer', '720', 'None', 0.414, -1.0, 0.430, -1.0)]

In [112]:
'''
DLinear
'''

dlinear_etth2 = [('p0', 'ETTh2', 'None', 'DLinear', '96', 'None', 0.288, -1.0, 0.353, -1.0),
                 ('p0', 'ETTh2', 'None', 'DLinear', '192', 'None', 0.383, -1.0, 0.417, -1.0),
                 ('p0', 'ETTh2', 'None', 'DLinear', '336', 'None', 0.447, -1.0, 0.465, -1.0),
                 ('p0', 'ETTh2', 'None', 'DLinear', '720', 'None', 0.605, -1.0, 0.551, -1.0)]

dlinear_weather = [('p0', 'weather', 'None', 'DLinear', '96', 'None', 0.175, -1.0, 0.237, -1.0),
                   ('p0', 'weather', 'None', 'DLinear', '192', 'None', 0.219, -1.0, 0.282, -1.0),
                   ('p0', 'weather', 'None', 'DLinear', '336', 'None', 0.265, -1.0, 0.318, -1.0),
                   ('p0', 'weather', 'None', 'DLinear', '720', 'None', 0.323, -1.0, 0.361, -1.0)]

In [118]:
'''
misstsm
'''

misstsm_etth2 = [('p0', 'ETTh2', 'None', 'MissTSM', '96', 'None', 0.255, -1.0, 0.350, -1.0),
               ('p0', 'ETTh2', 'None', 'MissTSM', '192', 'None', 0.234, -1.0, 0.345, -1.0),
               ('p0', 'ETTh2', 'None', 'MissTSM', '336', 'None', 0.316, -1.0, 0.397, -1.0),
               ('p0', 'ETTh2', 'None', 'MissTSM', '720', 'None', 0.305, -1.0, 0.389, -1.0)]

misstsm_ettm2 = [('p0', 'ETTm2', 'None', 'MissTSM', '96', 'None', 0.183, -1.0, 0.295, -1.0),
               ('p0', 'ETTm2', 'None', 'MissTSM', '192', 'None', 0.209, -1.0, 0.318, -1.0),
               ('p0', 'ETTm2', 'None', 'MissTSM', '336', 'None', 0.261, -1.0, 0.349, -1.0),
               ('p0', 'ETTm2', 'None', 'MissTSM', '720', 'None', 0.311, -1.0, 0.383, -1.0)]

misstsm_weather = [('p0', 'weather', 'None', 'MissTSM', '96', 'None', 0.164, -1.0, 0.227, -1.0),
               ('p0', 'weather', 'None', 'MissTSM', '192', 'None', 0.210, -1.0, 0.267, -1.0),
               ('p0', 'weather', 'None', 'MissTSM', '336', 'None', 0.254, -1.0, 0.292, -1.0),
               ('p0', 'weather', 'None', 'MissTSM', '720', 'None', 0.324, -1.0, 0.342, -1.0)]

In [114]:
all_lists = [simmtm_etth2, simmtm_ettm2, simmtm_weather, ptst_etth2, ptst_ettm2, ptst_weather,
             itransformer_etth2, itransformer_weather, autoformer_etth2, autoformer_weather,
            dlinear_etth2, dlinear_weather]

# Step 3: Iterate over the lists, convert each to a DataFrame, and append to the existing DataFrame
for tuples_list in all_lists:
    new_df = pd.DataFrame(tuples_list, columns=cols)
    df1 = pd.concat([df1, new_df], ignore_index=True)

In [116]:
df1

,Fraction,Dataset,Masking,TS_Model,Prediction_Length,Imputation_Model,MSE_Mean,MSE_STD,MAE_Mean,MAE_STD
0,p6,ETTh2,MCAR,DLinear,96,SAITS,0.267415,0.015955,0.336383,0.008934
1,p6,ETTh2,MCAR,DLinear,192,SAITS,0.377993,0.031653,0.409279,0.015572
2,p6,ETTh2,MCAR,DLinear,336,SAITS,0.449674,0.036940,0.460009,0.018715
3,p6,ETTh2,MCAR,DLinear,720,SAITS,0.618890,0.046420,0.550535,0.020014
4,p7,ETTh2,MCAR,DLinear,96,SAITS,0.253664,0.028532,0.326745,0.018440
...,...,...,...,...,...,...,...,...,...,...
785,p0,ETTh2,None,DLinear,720,None,0.605000,-1.000000,0.551000,-1.000000
786,p0,weather,None,DLinear,96,None,0.175000,-1.000000,0.237000,-1.000000
787,p0,weather,None,DLinear,192,None,0.219000,-1.000000,0.282000,-1.000000
788,p0,weather,None,DLinear,336,None,0.265000,-1.000000,0.318000,-1.000000


In [128]:
dfx=pd.read_csv('all_models_final.csv')

In [129]:
new_lists=[misstsm_etth2, misstsm_ettm2, misstsm_weather]

for tuples_list in new_lists:
    new_df = pd.DataFrame(tuples_list, columns=cols)
    dfx = pd.concat([dfx, new_df], ignore_index=True)

In [130]:
dfx

,Fraction,Dataset,Masking,TS_Model,Prediction_Length,Imputation_Model,MSE_Mean,MSE_STD,MAE_Mean,MAE_STD
0,p6,ETTh2,MCAR,DLinear,96,SAITS,0.267415,0.015955,0.336383,0.008934
1,p6,ETTh2,MCAR,DLinear,192,SAITS,0.377993,0.031653,0.409279,0.015572
2,p6,ETTh2,MCAR,DLinear,336,SAITS,0.449674,0.036940,0.460009,0.018715
3,p6,ETTh2,MCAR,DLinear,720,SAITS,0.618890,0.046420,0.550535,0.020014
4,p7,ETTh2,MCAR,DLinear,96,SAITS,0.253664,0.028532,0.326745,0.018440
...,...,...,...,...,...,...,...,...,...,...
797,p0,ETTm2,None,MissTSM,720,None,0.311000,-1.000000,0.383000,-1.000000
798,p0,weather,None,MissTSM,96,None,0.164000,-1.000000,0.227000,-1.000000
799,p0,weather,None,MissTSM,192,None,0.210000,-1.000000,0.267000,-1.000000
800,p0,weather,None,MissTSM,336,None,0.254000,-1.000000,0.292000,-1.000000


In [131]:
dfx.to_csv('all_models_finalissma.csv', index=0)

In [145]:
dfx[(dfx['TS_Model']=='PatchTST') & (dfx['Dataset']=='weather') & ~(dfx['Fraction']=='p0') & (dfx['Masking']=='MCAR')].to_csv('forecasting_errs_PTST.csv', index=False)